In [2]:
from glob import glob
import re
import os
import cv2
import numpy as np
from scipy.ndimage import map_coordinates 
import nibabel as nib
import matplotlib.pyplot as plt

In [3]:
dataset_path = "C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/New_Data_09_03_2025/Finsh_Seg/Results"
cases = glob(dataset_path+"/*")
print(cases)

['C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/New_Data_09_03_2025/Finsh_Seg/Results\\k2025_5']


In [4]:
# Step 1: Load PNG files into a list
def load_images(input_folder, type):
    file_list = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])
    slices = []
    for file in file_list:
        img = cv2.imread(os.path.join(input_folder, file), 0)
        img = img / img.max()
        if type != "img":
            vals = np.unique(img)
            img[img == vals[1]] = 255
            img[img == vals[2]] = 127
        slices.append(np.array(img))
    return slices[:37]  # Ensure only the first 36 images are loaded

In [5]:
import numpy as np

def cylindrical_volume(slices, angle_step, centroid=None, use_interpolation=True):
    # Assume each slice is a 2D numpy array with shape (height, width)
    height, width = slices[0].shape
    print(f"Slice dimensions: {(height, width)}")
    
    # Determine the center of the image.
    # If a centroid tuple is provided, use it. Otherwise, default to the image center.
    center_x = center_y = centroid
    
    # Use a square base dimension that is the maximum of width and height.
    base = width
    # Create the volume with dimensions (base, base, height).
    # The x-y plane is square to fully capture rotated coordinates.
    volume = np.zeros((base, base, height), dtype=slices[0].dtype)
    
    # Process each slice.
    for i, slice_img in enumerate(slices):
        theta = np.deg2rad(i * angle_step)
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)
        
        # Loop over pixel positions relative to the image center.
        for dx in range(-center_x, center_x):
            for dy in range(-center_y, center_y):
                # Compute the rotated coordinates for the horizontal offset (dx).
                # These determine where the pixel falls in the x-y plane of the volume.
                x_rot = int(base // 2 + dx * cos_theta)
                y_rot = int(base // 2 + dx * sin_theta)
                # Use the vertical offset (dy) to determine the z coordinate.
                z_pos = dy + center_y  # This will be in [0, height)
                
                # Ensure the computed indices are within the volume bounds.
                if 0 <= x_rot < base and 0 <= y_rot < base and 0 <= z_pos < height:
                    # Map the pixel from the slice into the volume.
                    volume[x_rot, y_rot, z_pos] = slice_img[dy + center_y, dx + center_x]
    
    return volume

In [6]:
# Step 3: Save as NIfTI
def save_nifti(volume, output_file):
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))
    nib.save(nifti_img, output_file)
    print(f"NIfTI file saved to {output_file}")

In [7]:
def create_nifti(case_folder, output_path, type, time, timestr):
    case = case_folder
    case = case_folder.replace("\\", "/").split("/")[-3]
    # print(case)

    slices = load_images(case_folder+f"/time{time:03d}",type)
    slices = np.array(slices)

    window = [np.shape(slices)[1],np.shape(slices)[1],np.shape(slices)[2]]

    centroid = np.shape(slices)[2]//2
    volume = cylindrical_volume(slices, 5,centroid ,False)

    output_folder = output_path+f"/{case}"
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    if type=="img":
        output_file = output_folder+f"/{case}_frame{timestr}.nii"
    elif type=="seg":
        output_file = output_folder+f"/{case}_frame{timestr}_seg.nii"
    save_nifti(volume, output_file)

    return window
    

In [14]:
output_path = "C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset"
for case in cases:
    # print(os.path.basename(case))
    with open(case + "/info.txt", 'r') as info:
        for line in info:
            if line.startswith("ES:"):
                ES = int(line.split(":", 1)[1].strip())
            elif line.startswith("ED:"):
                ED = int(line.split(":", 1)[1].strip())
    ES = int(ES)
    ED = int(ED)
    print(f"ES={ES}")
    print(f"ED={ED}")
    #Load ES Image
    window = create_nifti(case,output_path,"img" ,ES, "ES")
    #Load ES Mask
    create_nifti(case, output_path,"seg" ,ES, "ES")
    #Load ED Image
    create_nifti(case,output_path,"img" ,ED, "ED")
    #Load ED Mask
    create_nifti(case,output_path,"seg" ,ED, "ED")

    with open(output_path+f"/{os.path.basename(case)}/window.txt", "w") as f:
        f.write(" ".join(map(str, window))) 

ES=24
ED=4
Slice dimensions: (967, 953)
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset/Finsh_Seg/Finsh_Seg_frameES.nii
Slice dimensions: (967, 953)
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset/Finsh_Seg/Finsh_Seg_frameES_seg.nii
Slice dimensions: (967, 953)
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset/Finsh_Seg/Finsh_Seg_frameED.nii
Slice dimensions: (967, 953)
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset/Finsh_Seg/Finsh_Seg_frameED_seg.nii


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/haziq/OneDrive - Imperial College London/Directional Data/HAZIQ/NIFTI_dataset/k2025_5/window.txt'